## Dependencies

In [3]:
import gym
import random
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

## Creating Open AI gym env

In [2]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [3]:
actions

2

# Test env

In [7]:
episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done= False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode,score))
    env.close()

Episode: 1 Score: 15.0
Episode: 2 Score: 26.0
Episode: 3 Score: 18.0
Episode: 4 Score: 9.0
Episode: 5 Score: 29.0
Episode: 6 Score: 15.0
Episode: 7 Score: 33.0
Episode: 8 Score: 50.0
Episode: 9 Score: 49.0
Episode: 10 Score: 20.0
Episode: 11 Score: 32.0
Episode: 12 Score: 40.0
Episode: 13 Score: 61.0
Episode: 14 Score: 25.0
Episode: 15 Score: 41.0
Episode: 16 Score: 28.0
Episode: 17 Score: 17.0
Episode: 18 Score: 15.0
Episode: 19 Score: 14.0
Episode: 20 Score: 21.0


## Create deep learning model

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [27]:
model = build_model(states, actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## build agents with keras-rl

In [5]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [7]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [30]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 61s 6ms/step - reward: 1.0000
95 episodes - episode_reward: 103.789 [10.000, 359.000] - loss: 2.022 - mae: 19.337 - mean_q: 39.160

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 64s 6ms/step - reward: 1.0000
48 episodes - episode_reward: 209.646 [157.000, 289.000] - loss: 3.443 - mae: 40.855 - mean_q: 82.529

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 1.0000
44 episodes - episode_reward: 227.795 [169.000, 362.000] - loss: 2.875 - mae: 43.397 - mean_q: 87.361

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 68s 7ms/step - reward: 1.0000
43 episodes - episode_reward: 233.442 [152.000, 343.000] - loss: 1.801 - mae: 42.413 - mean_q: 85.308

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 1.0000
done

In [31]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 276.000, steps: 276
Episode 2: reward: 278.000, steps: 278
Episode 3: reward: 253.000, steps: 253
Episode 4: reward: 292.000, steps: 292
Episode 5: reward: 271.000, steps: 271
Episode 6: reward: 208.000, steps: 208
Episode 7: reward: 299.000, steps: 299
Episode 8: reward: 183.000, steps: 183
Episode 9: reward: 209.000, steps: 209
Episode 10: reward: 256.000, steps: 256
Episode 11: reward: 262.000, steps: 262
Episode 12: reward: 246.000, steps: 246
Episode 13: reward: 247.000, steps: 247
Episode 14: reward: 265.000, steps: 265
Episode 15: reward: 204.000, steps: 204
Episode 16: reward: 253.000, steps: 253
Episode 17: reward: 221.000, steps: 221
Episode 18: reward: 233.000, steps: 233
Episode 19: reward: 195.000, steps: 195
Episode 20: reward: 182.000, steps: 182
Episode 21: reward: 258.000, steps: 258
Episode 22: reward: 359.000, steps: 359
Episode 23: reward: 364.000, steps: 364
Episode 24: reward: 210.000, steps: 210
Episode 25: reward: 

In [33]:

_ = dqn.test(env, nb_episodes=50, visualize=True)

Testing for 50 episodes ...
Episode 1: reward: 216.000, steps: 216
Episode 2: reward: 244.000, steps: 244
Episode 3: reward: 215.000, steps: 215
Episode 4: reward: 166.000, steps: 166
Episode 5: reward: 244.000, steps: 244
Episode 6: reward: 234.000, steps: 234
Episode 7: reward: 343.000, steps: 343
Episode 8: reward: 210.000, steps: 210
Episode 9: reward: 269.000, steps: 269
Episode 10: reward: 230.000, steps: 230
Episode 11: reward: 324.000, steps: 324
Episode 12: reward: 309.000, steps: 309
Episode 13: reward: 264.000, steps: 264
Episode 14: reward: 205.000, steps: 205
Episode 15: reward: 324.000, steps: 324
Episode 16: reward: 218.000, steps: 218
Episode 17: reward: 221.000, steps: 221
Episode 18: reward: 184.000, steps: 184
Episode 19: reward: 295.000, steps: 295
Episode 20: reward: 245.000, steps: 245
Episode 21: reward: 328.000, steps: 328
Episode 22: reward: 210.000, steps: 210
Episode 23: reward: 330.000, steps: 330
Episode 24: reward: 222.000, steps: 222
Episode 25: reward: 3

## reloading model from memory

In [34]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [1]:
del model
del dqn
del env

NameError: name 'model' is not defined

In [9]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [10]:
dqn.load_weights('dqn_weights.h5f')

In [11]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 180.000, steps: 180
Episode 3: reward: 178.000, steps: 178
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
